In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_csv('./dataset_new.csv')
test_df = pd.read_csv('./dataset_test_new.csv')
df.head(5)

,LOTE,Fecha/hora inicio_preinoculo,Fecha/hora fin_preinoculo,línea 1,línea 2,línea 3,línea 1.1,línea 2.1,línea 3.1,línea 1.2,...,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2,media_PV,temperature_preinoculo,humedad_preinoculo,media_temp_bioreactor,media_ph_biorreactor,media_PO_biorreactor
0,23019,2023-03-19 05:00:00,2023-03-20 07:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1747.920,6.00,0.330522,20.157241,50.339050,29.689535,6.191826,24.355140
1,23020,2023-03-19 05:00:00,2023-03-20 07:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1676.160,6.56,0.426711,20.157241,50.339050,29.689535,6.191826,24.355140
2,23021,2023-03-20 05:00:00,2023-03-21 06:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1928.496,8.08,0.409436,20.173527,52.099099,29.439842,6.028036,25.462923
3,23022,2023-03-20 05:00:00,2023-03-21 06:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1782.800,5.92,0.080985,20.173527,52.099099,29.439842,6.028036,25.462923
4,23023,2023-03-26 05:00:00,2023-03-27 07:21:00,5.496,5.504,5.5200000000000005,28.32,27.92,32,1.0,...,26.56,20.88,1861.840,2.96,0.368708,20.251146,48.900195,29.442474,6.002998,23.396945


In [3]:
target = df.iloc[:,41]
target_test = test_df.iloc[:,41]
columns_to_drop = df.iloc[:, [0,1,2,13,14,12,22,23,24,30,31,32,33,38,41,42]].columns

df_dropped = df.drop(columns=columns_to_drop)
df_test_dropped = test_df.drop(columns=columns_to_drop)

In [4]:
def clean_and_convert(column):
    # Reemplazar ',' por '.' y eliminar el signo '+'
    column = column.str.replace(',', '.', regex=False)
    column = column.str.replace('+', '', regex=False)
    # Convertir a float, forzando errores a NaN
    return pd.to_numeric(column, errors='coerce')

# Aplicar la función a todas las columnas de tipo object


In [5]:
def data_nan(data):
    data.replace('N.A', np.nan, inplace=True)
    data['línea 3'] = data['línea 3'].astype(float)
    data['línea 3.1'] = data['línea 3.1'].astype(float)
    data['Viabilidad final cultivo_cultivo_final'] = data['Viabilidad final cultivo_cultivo_final'].astype(float)
    for column in data.select_dtypes(include=[np.number]).columns:
        mean_value = data[column].mean()
        print(mean_value)
        data[column].fillna(mean_value, inplace=True) 
    for col in data.select_dtypes(include=['object']).columns:
        data[col] = clean_and_convert(data[col])
    return data

In [8]:
data = data_nan(df_dropped)
data_test = data_nan(df_test_dropped)
data = data_nan(df_dropped)
data_test = data_nan(df_test_dropped)

5.468736
5.470035087719299
5.498217821782178
32.061440000000005
30.212631578947367
29.80118811881188
0.8188976377952756
0.6456692913385826
0.4173228346456693
150.5514503816794
15.36064
26.362137404580157
99092268.70229007
13.696641221374044
20.1720479825518
44926560.913380995
150.5514503816794
15.36064
26.362137404580157
99092268.70229007
1.0
81.55658914728681
17.380763358778626
74.06106870229007
168240203.56234095
30.28740157480315
23.35516129032258
0.36618489715278335
20.16984734592939
47.94785228808695
29.45899697760463
6.965512947242943
27.171243680776026
5.463030303030303
5.440727272727273
5.445793103448277
28.007272727272728
28.21090909090909
27.111724137931038
0.8181818181818182
0.8484848484848485
0.24242424242424243
159.48533333333336
15.098181818181818
25.413333333333334
111985454.54545455
13.632727272727273
21.534343434343434
70642149.49494952
159.48533333333336
15.098181818181818
25.413333333333334
111985454.54545455
1.0
82.53241379310346
18.30545454545455
78.84363636363636


In [9]:
data.isna().sum()

línea 1                                   0
línea 2                                   0
línea 3                                   0
línea 1.1                                 0
línea 2.1                                 0
línea 3.1                                 0
línea 1.2                                 0
línea 2.2                                 0
línea 3.2                                 0
Volumen de cultivo                        0
Turbidez inicio cultivo                   0
Turbidez final culttivo                   0
Viabilidad final cultivo                  0
Cantidad entregada                        0
Turbidez                                  0
Viabilidad                                0
Volumen de cultivo_inoculo                0
Turbidez inicio cultivo_inoculo           0
Turbidez final culttivo_inoculo           0
Viabilidad final cultivo_inoculo          0
Orden en el encadenado                    0
Volumen de inóculo utilizado              0
Turbidez inicio cultivo_cultivo_

In [10]:
# Definir características (X) y objetivo (y)
X = data
X_test_final = data_test
y = target

In [11]:

# Dividir el dataset en entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% entrenamiento
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% validación, 20% prueba

In [12]:

# Crear y entrenar el modelo
model = LinearRegression()

In [13]:

model.fit(X_train, y_train)

LinearRegression()

In [14]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),                  # Escalado de las características
    ('regressor', RandomForestRegressor(random_state=42))  # Modelo de regresión
])

In [15]:


# Definir los hiperparámetros a ajustar
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

In [16]:

# Realizar la búsqueda en cuadrícula (Grid Search) para encontrar los mejores parámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Predecir en el conjunto de validación
y_val_pred = grid_search.predict(X_val)

# Evaluar el modelo
mse_val = mean_squared_error(y_val, y_val_pred)
print("Mean Squared Error en el conjunto de validación:", np.sqrt(mse_val))

# Predecir en el conjunto de prueba
y_test_pred = grid_search.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
print("Mean Squared Error en el conjunto de prueba:", np.sqrt(mse_test))

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END regressor__max_depth=None, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=50; total time=   0.1s
[CV] END regressor__max_depth=None, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=50; total time=   0.0s
[CV] END regressor__max_depth=None, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=50; total time=   0.1s
[CV] END regressor__max_depth=None, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=50; total time=   0.1s
[CV] END regressor__max_depth=None, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=50; total time=   0.0s
[CV] END regressor__max_depth=None, regressor__min_samples_leaf=1, regressor__min_samples_split=2, regressor__n_estimators=100; total time=   0.2s
[CV] END regressor__max_depth=None, regressor__min_samples_l

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import numpy as np

# Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),               # Escalado de las características
    ('regressor', XGBRegressor(random_state=42))  # Modelo XGBoost
])

# Definir los hiperparámetros a ajustar
param_distributions = {
    'regressor__n_estimators': [100, 200, 300, 500],
    'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'regressor__max_depth': [3, 5, 7, 10],
    'regressor__min_child_weight': [1, 3, 5],
    'regressor__subsample': [0.6, 0.8, 1.0],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0]
}

# Realizar la búsqueda aleatoria (Random Search) para encontrar los mejores parámetros
random_search = RandomizedSearchCV(pipeline, param_distributions, cv=5, 
                                   scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Entrenar el modelo
random_search.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:", random_search.best_params_)

# Predecir en el conjunto de validación
y_val_pred = random_search.predict(X_val)

# Evaluar el modelo en el conjunto de validación
mse_val = mean_squared_error(y_val, y_val_pred)
print("Root Mean Squared Error en el conjunto de validación:", np.sqrt(mse_val))

# Predecir en el conjunto de prueba
y_test_pred = random_search.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
print("Root Mean Squared Error en el conjunto de prueba:", np.sqrt(mse_test))


NameError: name 'X_train' is not defined

In [59]:
y_final_pred = random_search.predict(X_test_final)

# Imprimir las predicciones para el dataset final
print("Predicciones para el conjunto data_test_final:", y_final_pred)

Predicciones para el conjunto data_test_final: [1632.5325 1594.293  1592.1426 1768.6703 1733.5317 1730.9021 1590.4525
 1579.6803 1579.6803 1394.9207 1379.5828 1379.5828 1701.0398 1710.4081
 1695.2001 1584.1467 1589.3624 1574.2567 1439.7001 1445.3746 1445.5474
 1446.085  1445.6285 1317.2355 1325.8512 1329.9785 1330.516  1330.0596
 1929.8623 1906.2745 1926.0521 1925.169  1925.4971 1905.2246 1673.219
 1663.9357 1663.8682 1662.985  1664.1024 1627.2231 2168.9497 2157.147
 2155.6838 2156.1875 1721.7982 1768.3978 1768.3978 1519.6267 1485.1937
 1485.1937 1445.7439 1448.6152 1448.6152 1572.8196 1569.1792 1569.9873
 1690.0575 1664.6311 1676.935  1685.8677 1685.806  1685.2588 1679.7036
 1762.1995 1759.8208 1733.6677 1723.032  1759.3733 1751.8849 1714.9634
 1716.4    1826.1714 1826.5062 1822.1272 1645.8364 1616.468  1612.1323
 1588.6622 1575.8914 1572.527  1834.4999 1819.8738 1837.439  1839.5688
 1651.6786 1628.4309 1628.371  1502.8152 1482.3558 1482.2959 1464.2433
 1455.9564 1455.9564 1455.9564 1

In [14]:
X.isna().sum()

línea 1                                   0
línea 2                                   0
línea 3                                   0
línea 1.1                                 0
línea 2.1                                 0
línea 3.1                                 0
línea 1.2                                 0
línea 2.2                                 0
línea 3.2                                 0
Volumen de cultivo                        0
Turbidez inicio cultivo                   6
Turbidez final culttivo                   0
Viabilidad final cultivo                  0
Cantidad entregada                        0
Turbidez                                  0
Viabilidad                                0
Volumen de cultivo_inoculo                0
Turbidez inicio cultivo_inoculo           6
Turbidez final culttivo_inoculo           0
Viabilidad final cultivo_inoculo          0
Orden en el encadenado                    0
Volumen de inóculo utilizado              0
Turbidez inicio cultivo_cultivo_